## 평가지표

+ 정확도  
+ 오차행렬  
+ 정밀도  
+ 재현율  
+ F1 스코어  
+ ROC, AUC  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

# 정확도
+ 실제 데이터에서 예측 데이터가 얼만가 같은지 판단하는 지표

In [2]:
from sklearn.base import BaseEstimator

class mydummyclassifier(BaseEstimator):
    def fit(self,X,y=None):
        pass
    def predict(self,X):
        pred=np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i]==1:
                pred[i]=0
            else:
                pred[i]=1
            
        return pred

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


#원본 데이터 재로딩,데이터 가공, 학습 데이터/테스트 데이터 분할

data=pd.read_csv('titanic.csv')
y=data['Survived']
x=data.drop('Survived',axis=1)
#x=transform_features(x)
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

myclf=mydummyclassifier()
myclf.fit(X_train,y_train)

prediction=myclf.predict(X_test)
print(f'mydummy의 정확도는 {round(accuracy_score(y_test,prediction),4)}')

mydummy의 정확도는 0.3855


------------------------------

#### MNIST 데이터셋을 변환해 정확도 분류해보기

In [4]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


class myfakeclassifier(BaseEstimator):
    def fit(self,X,y):
        pass
    def predict(self,X):
        return np.zeros((len(X),1),dtype=bool)
    
digit=load_digits()

y=(digit.target==7).astype(int)
X_train,X_test,y_train,y_test=train_test_split(digit.data,y,random_state=11)

In [5]:
# 불균형 레이블 분포도 확인

print(f'레이블 테스트 크기 {y_test.shape}')
print(f'테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

#dummyclassifier로 평가

fakeclf=myfakeclassifier()
fakeclf.fit(X_train,y_test)
predict=fakeclf.predict(X_test)
print(f'모든 예측을 0으로 하여도 정확도는 {accuracy_score(y_test,predict)}')

레이블 테스트 크기 (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는 0.9


##### 단순히 predict() 결과를 모두 0으로 반환해도 성능이 90%나온다.
##### 불균형한 데이터셋에서 정확도 지표는 옳지 못한 선택임을 알 수있으며
##### 이를 위해 다른 여러 지표를 사용한다.

# 오차 행렬

+ 모델이 얼마나 헷갈리고 있는지도 함께 보여주는 지표
+ 예측 오류가 얼마인지, 어떤 오류가 발생하는지도 나타냄

In [6]:
from IPython.display import Image

# ![title](matrix.png)  마크다운 형식으로 이미지 삽입

![title](matrix.png)

In [7]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,predict)

array([[405,   0],
       [ 45,   0]], dtype=int64)

![title](matrix_2.png)

# 정밀도와 재현율

+ Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가지표
    + 정밀도 = TP/(FP+TP)
    + 재현율 = TP/(FN+TP)

In [9]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix

In [44]:
def get_clf_eval(y_test,pred):
    confusion=confusion_matrix(y_test,pred)
    accuracy=accuracy_score(y_test,pred)
    precision=precision_score(y_test,pred)
    recall=recall_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print(f'정확도: {round(accuracy,4)}, 정밀도: {round(precision,4)}, 재현율: {round(recall,4)}')

In [33]:
from sklearn import preprocessing
def fillna(data):
    data['Age'].fillna(data['Age'].mean(),inplace=True)
    data['Cabin'].fillna('N',inplace=True)
    data['Embarked'].fillna('N',inplace=True)
    data['Fare'].fillna(0,inplace=True)
    
    return data

def drop(data):
    data.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return data

def format_feature(data):
    data['Cabin']=data['Cabin'].str[:1]
    features=['Cabin','Sex','Embarked']
    for i in features:
        le=preprocessing.LabelEncoder()
        le=le.fit(data[i])
        data[i]=le.transform(data[i])
    return data

#함수들 호출해 전처리 수행

def transform(data):
    data=fillna(data)
    data=drop(data)
    data=format_feature(data)
    return data

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

data=pd.read_csv('titanic.csv')
y=data['Survived']
x=data.drop('Survived',axis=1)
x=transform(x)

X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=11)

lr_clf=LogisticRegression()
lr_clf.fit(X_train,y_train)
pred=lr_clf.predict(X_test)

get_clf_eval(y_test,pred)

오차 행렬
[[108  10]
 [ 14  47]]
정확도: 0.8659, 정밀도: 0.8246, 재현율: 0.7705


#### 매번 데이터셋을 전처리 할 순 없으니 x,y를 합친 preprocessing을 내보내서 활용할 예정..
 + x['Survived']=y
 + x.to_csv('preprocessing.csv',index=False)